In [2]:
import pandas as pd
import numpy as np

# Rebate Data

In [132]:
# Create a list of all post codes
df = pd.read_csv('static/data/australian_postcodes.csv')
rebate_df = pd.DataFrame(df.postcode.unique(), columns={'postcode'})
rebate_df['postcode'].astype('int64')
rebate_df

,postcode
0,200
1,800
2,801
3,804
4,810
...,...
3163,9010
3164,9013
3165,9015
3166,9464


In [135]:
df = pd.read_csv('rebate.csv')
df[['Postcode_from', 'Postcode_to']].astype('int64')


zone_list = []
rating_list = []
annual_prod = []
rebate = []

# Rebate amount is equal to number of certificates * rebate rate * years left in the program
# 1 MWh = 1 certificate. Rate of production dependent on zone
years_left = 10          
rebate_rate = 35    # $per certificate


def find_data(code):
    for index, row in df.iterrows():
            postcode = row['Postcode_to'].astype('int64')
            if (code <= postcode):
                zone = row['Zone']
                zone_list.append(zone)
                
                rating_list.append(row['Rating'])
                if zone == 1:
                    annual_prod.append(10.7)
                    rebate.append(10.7 * years_left * rebate_rate)
                elif zone ==2:
                    annual_prod.append(10.1)
                    rebate.append(10.1 * years_left * rebate_rate)
                elif zone ==3:
                    annual_prod.append(9.1)
                    rebate.append(9.1 * years_left * rebate_rate)
                elif zone ==4:
                    annual_prod.append(7.8)
                    rebate.append(7.8 * years_left * rebate_rate)
                
                break
                

for index, row in rebate_df.iterrows():
    code = row['postcode']
    find_data(code)
    
            
        

In [136]:
# Create final rebate dataframe
rebate_df['zone'] = zone_list
rebate_df['rating'] = rating_list
rebate_df['annual_prod'] = annual_prod
rebate_df['rebate'] = rebate
rebate_df


,postcode,zone,rating,annual_prod,rebate
0,200,3.0,1.382,9.1,3185.0
1,800,2.0,1.536,10.1,3535.0
2,801,2.0,1.536,10.1,3535.0
3,804,2.0,1.536,10.1,3535.0
4,810,2.0,1.536,10.1,3535.0
...,...,...,...,...,...
3163,9010,3.0,1.382,9.1,3185.0
3164,9013,3.0,1.382,9.1,3185.0
3165,9015,3.0,1.382,9.1,3185.0
3166,9464,3.0,1.382,9.1,3185.0


# Income Data

In [149]:
df = pd.read_csv("static/data/Income.csv")

# Remove all non-numeric and Nan values from the postcode column
postcodes = df['Postcode'].apply(pd.to_numeric, errors='coerce')
df = df.drop('Postcode', axis=1).join(postcodes)
df.dropna(axis=0, subset={'Postcode'}, inplace=True )

# Convert postcode to integer
df['Postcode'] = df['Postcode'].astype('int64')
df

,Taxable status,State/ Territory1,Number of individuals\nno.,Taxable income or loss3\nno.,Taxable income or loss3\n$,Tax on taxable income\nno.,Tax on taxable income\n$,Medicare levy\nno.,Medicare levy\n$,Medicare levy surcharge\nno.,...,Gross interest\n$,Dividends unfranked\nno.,Dividends unfranked\n$,Dividends franked\nno.,Dividends franked\n$,Dividends franking credit\nno.,Dividends franking credit\n$,Total Income or Loss3\nno.,Total Income or Loss3\n$,Postcode
0,Non Taxable,ACT,"1,023",991,"11,112,598",277,"509,577",9,"15,090",1,...,"1,340,572",119,"56,457",342,"1,226,487",341,"522,350","1,002","12,656,277",2600
1,Non Taxable,ACT,971,924,"8,433,619",126,"127,817",5,"4,506",0,...,"384,963",27,"20,004",70,"159,286",70,"67,916",924,"8,826,706",2601
2,Non Taxable,ACT,"4,034","3,904","47,209,582",953,"1,116,312",15,"11,475",0,...,"2,682,890",282,"113,053",862,"1,700,107",861,"724,882","3,912","50,054,366",2602
3,Non Taxable,ACT,"1,033",983,"11,225,453",251,"303,684",6,"3,579",0,...,"1,302,103",91,"29,431",301,"909,047",300,"389,065",997,"13,385,732",2603
4,Non Taxable,ACT,"1,188","1,135","14,090,650",297,"498,559",12,"11,683",2,...,"890,118",89,"33,409",295,"493,955",294,"211,243","1,152","15,200,174",2604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5380,Taxable,WA,50,50,"5,168,088",50,"1,426,362",44,"99,226",0,...,"36,546",2,436,10,"83,085",10,"35,439",50,"5,352,038",6964
5381,Taxable,WA,56,56,"12,857,689",56,"4,775,265",48,"253,501",4,...,"28,207",10,"17,190",15,"132,719",15,"54,861",56,"13,077,112",6979
5382,Taxable,WA,68,68,"6,262,209",68,"1,586,421",67,"122,884",1,...,"40,529",6,"2,238",13,"60,175",13,"24,521",68,"6,550,762",6981
5383,Taxable,WA,44,44,"4,581,968",44,"1,302,007",41,"89,932",0,...,"80,759",4,"1,081",11,"982,891",11,"373,798",44,"4,729,090",6984


In [151]:
# Remove the commas from the income values
df = df.replace(',','', regex=True)

# Convert income values from string to integer
df[['Total Income or Loss3\n$', 'Number of individuals\nno.', 'Salary or wages\n$']] = df[['Total Income or Loss3\n$', 'Number of individuals\nno.', 'Salary or wages\n$']].astype('int64')

# Add average income column (income / number of individuals)
df['Average_total'] = df['Total Income or Loss3\n$']/ df['Number of individuals\nno.']
df['Average_salary'] = df['Salary or wages\n$']/ df['Number of individuals\nno.']
df['Average_total']

0        12371.727273
1         9090.325438
2        12408.122459
3        12958.114230
4        12794.759259
            ...      
5380    107040.760000
5381    233519.857143
5382     96334.735294
5383    107479.318182
5384     99311.566038
Name: Average_total, Length: 5366, dtype: float64

In [153]:
# Seperate out the taxable and non-taxable income
taxable = df[df['Taxable status'] == 'Taxable']
non_taxable = df[df['Taxable status'] == 'Non Taxable']

# Concatenate the two dataframes and then, add the two together, grouped by postcode
total = pd.concat([taxable, non_taxable]).groupby('Postcode', as_index=False)[["Average_total", 'Average_salary']].sum()

# Fix the formatting and datatypes
total['Average_total'].round(0)
income_df = total.astype('int64')
income_df

,Postcode,Average_total,Average_salary
0,800,98643,88445
1,801,120690,85064
2,804,108716,89647
3,810,94079,78351
4,811,119181,90769
...,...,...,...
2678,7468,81527,61077
2679,7469,124341,70848
2680,7470,105301,90135
2681,8007,147200,105617
